## section 1: derivatives

In [1]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [6]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf(a, b, c):
  return [-3*a**2-0.5/a**0.5, cos(3*b)*3+2.5*b**1.5, 1.0/c**2] # todo, return [df/da, df/db, df/dc]

# expected answer is the list of 
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")

OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [11]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

# -----------
h = 0.0001
f_a = f(a,b,c)
f_ah = f(a+h,b,c)

f_b = f(a,b,c)
f_bh = f(a,b+h,c)

f_c = f(a,b,c)
f_ch = f(a,b,c+h)

numerical_grad = [(f_ah-f_a)/h,(f_bh-f_b)/h,(f_ch-f_c)/h] # TODO
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")

WRONG! for dim 0: expected -12.353553390593273, yours returns -12.354148981312818
WRONG! for dim 1: expected 10.25699027111255, yours returns -0.4698534461411441
OK for dim 2: expected 0.0625, yours returns 0.06249843753636242


In [17]:
# there is an alternative formula that provides a much better numerical 
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

# -----------

h = 0.0001

# Symmetric derivative for a
f_a_minus_h = f(a - h, b, c)
f_a_plus_h = f(a + h, b, c)
grad_a = (f_a_plus_h - f_a_minus_h) / (2 * h)

# Symmetric derivative for b
f_b_minus_h = f(a, b - h, c)
f_b_plus_h = f(a, b + h, c)
grad_b = (f_b_plus_h - f_b_minus_h) / (2 * h)

# Symmetric derivative for c
f_c_minus_h = f(a, b, c - h)
f_c_plus_h = f(a, b, c + h)
grad_c = (f_c_plus_h - f_c_minus_h) / (2 * h)

# Combine the gradients into a list
numerical_grad2 =  [grad_a, grad_b, grad_c]
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")

OK for dim 0: expected -12.353553390593273, yours returns -12.353553400705763
WRONG! for dim 1: expected 10.25699027111255, yours returns -0.46997744212795567
OK for dim 2: expected 0.0625, yours returns 0.0625000000464837


## section 2: support for softmax

In [30]:
# Value class starter code, with many functions taken out
from math import exp, log

class Value:

    def __init__(self, data, _children=(), _op='', label=''):
        """Initializes the object of Value class."""
        # Attribute for storing values
        self.data = data
        # Attribute for storing computed gradients
        self.grad = 0.0
        # Attribute for storing backpropagation function
        self._backward = lambda: None
        # Attribute for storing children nodes in the computational graph
        self._prev = set(_children)
        # Attribute for storting a label for the mathematical operation
        self._op = _op
        # Attribute for storting a label of the Value instance
        self.label = label
        
    def __repr__(self):
        """Returns string representation of the object."""
        
        return f"Value(data={self.data})"
    def __add__(self, other):
        """Adds values of two instances together."""
        # Convering a number to Value object if needed
        other = other if isinstance(other, Value) else Value(other)
        # Addition operation
        out = Value(self.data + other.data, (self, other), '+')
        # Computes gradients for the addition operation
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        
        return out
    
    def __radd__(self, other):
        """Implements reversal addition operation."""
        
        return self + other
    
    def __mul__(self, other):
        """Multiplies values of two instances together."""
        # Convering a number to Value object if needed
        other = other if isinstance(other, Value) else Value(other)
        # Multiplication operation
        out = Value(self.data * other.data, (self, other), "*")
        # Computes gradients for the multiplication operation
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        
        return out
    
    def __rmul__(self, other):
        """Implements reversal multiplication operation."""
        
        return self * other
    def __neg__(self):
        """Implements taking a negative value."""
        
        return self * (-1)
    
    def __pow__(self, other):
        """Computes value of one instance to the power of the value of another instance."""
        # Making sure that the values supplied are integer or floating point number
        assert isinstance(other, (int, float)), "Only integer and floating point powers are supported"
        # Power operation
        out = Value(self.data**other, (self,), f'**{other}')
        # Computes gradient for the power operation
        def _backward():
            self.grad += other * (self.data ** (other - 1)) * out.grad
        out._backward = _backward
        
        return out
    
    def __truediv__(self, other):
        """Implements a division operation."""
        
        return self * other**(-1)
    
    def __sub__(self, other):
        """Implements a subtraction operation."""
        
        return self + (-other)
    
    def __rsub__(self, other):
        """Implements a reversal subtraction operation"""
        
        return self + (-other)
    def exp(self):
        """Implements the operation of taking an exponent."""
        # Exponent of some power operation
        x = self.data
        out = Value(exp(x), (self, ), 'exp')
        # Computing gradient for the exponent operation
        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward
        
        return out
    
    def log(self):
        """Implements an operation of taking a natural logarithm."""
        # Natural logarithm operation
        x = self.data
        out = Value(log(x), (self, ), 'log')
        # Computing gradient for the natural logarithm operation
        def _backward():
            self.grad += (1/x) * out.grad
        out._backward = _backward
        
        return out

    def backward(self):
        """Implements a backpropagation via topological sorting."""
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

In [31]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
    """Implements a Softmax activation function."""
    # Exponentiating all inputs
    counts = [logit.exp() for logit in logits]
    # Computing probabilities
    denominator = sum(counts)
    out = [c / denominator for c in counts]
    
    return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
    ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
    print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")

2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


In [33]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch
import torch.nn.functional as F

In [34]:
# Defining a PyTorch tensor
logit_tensor = torch.Tensor([0.0, 3.0, -2.0, 1.0]).double()

# Enabling computing gradients
logit_tensor.requires_grad = True

logit_tensor

tensor([ 0.,  3., -2.,  1.], dtype=torch.float64, requires_grad=True)

In [35]:
# Computing an NLL loss (forward pass)
loss = -F.log_softmax(logit_tensor, dim=-1)[3]

In [37]:
# Backpropagation (backward pass)
loss.backward()

In [38]:
print(loss.data.item())

2.1755153626167143


In [39]:
logit_tensor.grad

tensor([ 0.0418,  0.8390,  0.0057, -0.8865], dtype=torch.float64)